# [SEMES] - 볼트 이상진단 - RegNet model

#### model 학습을 진행할 때 필요한 라이브러리 import

In [ ]:
import os
# 파이토치의 핵심 패키지(모델 구성 및 학습 등을 수행할 수 있는 기능을 제공)
import torch
# PyTorch에서 제공하는 신경망 모듈
import torch.nn as nn
# 학습에 사용되는 최적화 알고리즘
import torch.optim as optim
# PyTorch에서 이미지 데이터 처리와 관련된 함수와 모델들을 제공
from torchvision import models, datasets
# transforms 모듈은 데이터 전처리를 위한 함수들을 제공
import torchvision.transforms as transforms
# torchsummary는 PyTorch 모델의 요약 정보를 출력하는 라이브러리 / summary(model)로 요청
from torchsummary import summary
# DataLoader를 이용하여 데이터셋에서 미니배치(minibatch)를 추출 
from torch.utils.data import DataLoader
# 시간과 관련된 함수를 제공
import time


# 그래프 - 데이터 시각화를 위해 그래프나 도형을 화면에 출력해줌
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torchvision

# 컨퓨전 매트릭스 - sklearn.metrics == 평가 지표를 계산하는 패키지 
# 컨퓨전 매트릭스 - classification_report == 분류 모델을 평가한 후 각 클래스마다 지표를 출력(정확도(Accuracy)0/밀도(Precision)/재현율(Recall)/F1-점수(F1-Score)/지원 개수(Support)
# f1 score - f1_score == 분류 모델을 평가하는 지표- 정밀도(precision)와 재현율(recall)의 조화평균(정확도와 유사하지만 불균형한 데이터에서도 적절한지 평가할 수 있는 지표가 됨)
from sklearn.metrics import classification_report, f1_score

# 컨퓨전 매트릭스 - 데이터 시각화 라이브러리
import seaborn as sns

# 평가코드
from PIL import Image
import torch.nn.functional as F

#### GPU 사용을 위한 설정

In [ ]:
# GPU가 사용 가능한 경우 cuda 사용 / GPU가 사용 불가능한 경우 CPU로 초기화하여 CPU 사용
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Using cuda")
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU instead.")

#### 데이터 전처리

In [ ]:
# 데이터 분할
# !pip install split-folders tqdm

import splitfolders  
# train : test  8:2 비율로 나눠서 새로운 경로에 저장 
splitfolders.ratio('./semes_bolt1', output="./semes_bolt", seed=1, ratio=(0.8, 0.2)) 

In [ ]:
# 학습데이터 전처리
train_transform = transforms.Compose([
    # 해상도를 (224,224)로 맞춰준다 (a fixed resolution of 224×224 is best, even at higher flops : [논문]Designing Network Design Spaces - [저자]Facebook AI Research (FAIR))
    transforms.Resize((224, 224)),
    # 이미지를 좌우로 뒤집어서 데이터 증강(augmentation)을 수행(확률을 높여준)
    transforms.RandomHorizontalFlip(),
    # 이미지를 PyTorch의 Tensor로 변환
    transforms.ToTensor(),
    # 흑백 이미지이기 때문에 1개의 채널을 정규화(흑백이미지는 보통 (평균 : 0.5 / 표준편차 : 0.5)로 정규화)
    # transforms.Normalize([0.5], [0.5])
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

])

# 테스트데이터 전처리
test_transform = transforms.Compose([
    # 해상도를 (224,224)로 맞춰준다
    transforms.Resize((224, 224)),
    # 이미지를 PyTorch의 Tensor로 변환
    transforms.ToTensor(),
    # 흑백 이미지이기 때문에 1개의 채널을 정규화
    # transforms.Normalize([0.5], [0.5])
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

])

# 데이터가 저장된 경로
data_dir = './semes_bolt'
print(os.path.join(data_dir, 'train'))

# 데이터가 저장된 경로에서 ImageFolder를 이용하여 이미지 데이터셋을 전처리한 후 로드(transforms_*==전처리 수행)
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), train_transform)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'val'), test_transform)

# DataLoader를 이용 / 데이터셋에서 미니배치(minibatch)를 추출 
# (batch_size==미니배치의 크기 / shuffle==데이터셋을 섞을지 여부 / num_workers==데이터셋을 불러올 때 사용할 프로세스 수)
train_loader = DataLoader(train_datasets, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_datasets, batch_size=32, shuffle=True, num_workers=4)

# 수행 결과를 출력
print('학습 데이터셋 크기:', len(train_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

# 학습된 클래스 이름과 수행 결과를 출력
class_names = train_datasets.classes
print('클래스:', class_names)

In [ ]:
def imshow(input, title):
    # 이미지로 변환해주는 matplotlib(ply)는 numpy 객체만 받을 수 있기 때문에 torch.Tensor를 numpy 객체로 변환
    # pytorh는 (채널, 높이, 너비)의 순서를 갖기 때문에 transpose((1, 2, 0))를 통해 (높이, 너비, 채널)로 순서를 바꿔준다
    input = input.numpy().transpose((1, 2, 0))
    # 데이터셋에서 계산된 평균과 표준편차
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    # mean = np.array([0.5])
    # std = np.array([0.5])
    # 정규화된 이미지 데이터에 평균을 더하고 표준편차를 곱하여 정규화를 해제
    input = std * input + mean
    # 이미지 데이터의 픽셀 값이 0보다 작거나 1보다 클 경우, 0 또는 1로 대체
    input = np.clip(input, 0, 1)
    # matplotlib를 이용하여 이미지 데이터를 출력
    plt.imshow(input)
    # 이미지의 타이틀을 설정
    plt.title(title)
    # 이미지를 보여줌
    plt.show()


# train_dataloader(학습 데이터)를 iterator(배치 단위)로 불러오기
iterator = iter(train_loader)

# iterator에서 배치 불러오기
inputs, classes = next(iterator)
# 입력 이미지를 격자 형태로 만들기
out = torchvision.utils.make_grid(inputs)
# imshow==정규화를 해제하고 이미지를 시각화 / title==현재 배치의 정답 클래스
imshow(out, title=[class_names[x] for x in classes])

#### RegNet_Y_1.6 모델 불러오기

In [ ]:
# RegNetY-16GF 모델 정의
model = models.regnet_y_1_6gf(pretrained=True)

# 불러온 네트워크 모델의 출력 뉴런 수를 저장
num_features = model.fc.in_features
# 새로운 Fully Connected 레이어 추가
model.fc = nn.Linear(num_features, 4)

# GPU를 사용하기 위해 모델을 CUDA 디바이스로 보냄
model.to(device)

# 손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)

#### DataSet 이미지 학습

In [ ]:
# 학습 epochs 설정
num_epochs = 100

# epoch에 따른 손실 값과 정확도를 저장하는 리스트
train_loss_list = []
train_acc_list = []
test_loss_list = []
test_acc_list = []

In [ ]:
# 현재시간 저장
start_time = time.time()

# 설정한 epochs 만큼 반복
for epoch in range(num_epochs):

    # Train --------------------------------------------------------------------
    # 모델을 학습모드로 설정
    model.train()
    # 현재까지 누적된 손실을 저장할 변수 초기화
    train_loss = 0
    # 현재까지 맞춘 이미지의 수를 저장할 변수 초기화
    train_correct = 0
    # 현재까지 학습한 이미지 수를 저장할 변수 초기화
    train_cnt = 0

    # 배치 단위로 나눈 학습 데이터 순회하며 불러와서
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        # 입력 이미지, 라벨 정보를 GPU를 사용하기 위해 to.device()사용
        inputs, labels = inputs.to(device), labels.to(device)
        # 학습 전, 이전 학습에서 계산된 gradient 값을 0으로 초기화
        optimizer.zero_grad()
        # 모델에 이미지 학습 (forward propagation이 이루어지며, 모델은 입력을 받아 출력값을 계산)
        outputs = model(inputs)
        # 모델이 예측한 출력값(outputs)과 실제 정답(labels)을 비교하여 손실 값을 계산
        loss = criterion(outputs, labels)
        # 손실 값의 gradient를 계산(backward propagation이 이루어지며, 손실 함수를 모델의 출력값으로 미분한 gradient 값을 계산)
        loss.backward()
        # optimization - 계산된 gradient 값을 이용하여 모델의 파라미터를 업데이트
        optimizer.step()

        # 현재 배치에서 계산된 손실 값을 train_loss에 더함
        train_loss += loss.item()
        # 출력값(outputs) 중에서 가장 큰 값(_)과 그 값이 존재하는 인덱스(predicted)를 반환
        _, predicted = outputs.max(1)
        # 배치에 포함되어 학습한 이미지의 수를 train_cnt에 더함
        train_cnt += labels.size(0)
        # 현재 배치에서 맞춘 개수를 train_correct에 더함
        train_correct += predicted.eq(labels).sum().item()
    
    # epoch 단위로 평균 손실 값과 정확도를 계산
    train_loss /= len(train_loader)
    train_acc = 100 * train_correct / train_cnt

    # 계산한 평균 손실 값과 정확도를 리스트에 추가
    train_loss_list.append(train_loss)
    train_acc_list.append(train_acc)

    # Test ---------------------------------------------------------------------
    # 모델을 평가 모드로 설정
    model.eval()
    # 현재까지 누적된 손실을 저장할 변수 초기화
    test_loss = 0
    # 현재까지 맞춘 이미지의 수를 저장할 변수 초기화
    test_correct = 0
    # 현재까지 학습한 이미지 수를 저장할 변수 초기화
    test_cnt = 0

    # 모델의 gradient가 필요하지 않아 파라미터에 대한 업데이트를 수행하지 않으면서 forward propagation을 수행해 시간 단축
    with torch.no_grad():
         # 배치 단위로 나눈 테스트 데이터 순회하며 불러와서
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            # 입력 이미지, 라벨 정보를 GPU를 사용하기 위해 to.device()사용
            inputs, labels = inputs.to(device), labels.to(device)
            # 모델에 이미지 학습 (forward propagation이 이루어지며, 모델은 입력을 받아 출력값을 계산)
            outputs = model(inputs)
            # 모델이 예측한 출력값(outputs)과 실제 정답(labels)을 비교하여 손실 값을 계산
            loss = criterion(outputs, labels)

            # 현재 배치에서 계산된 손실 값을 test_loss에 더함
            test_loss += loss.item()
            # 출력값(outputs) 중에서 가장 큰 값(_)과 그 값이 존재하는 인덱스(predicted)를 반환
            _, predicted = outputs.max(1)
            # 배치에 포함되어 학습한 이미지의 수를 test_cnt에 더함
            test_cnt += labels.size(0)
            # 현재 배치에서 맞춘 개수를 test_correct에 더함
            test_correct += predicted.eq(labels).sum().item()
    
    # epoch 단위로 평균 손실 값과 정확도를 계산
    test_loss /= len(test_loader)
    test_acc = 100 * test_correct / test_cnt
    # 계산한 평균 손실 값과 정확도를 리스트에 추가
    test_loss_list.append(test_loss)
    test_acc_list.append(test_acc)

    # epoch마다 정확도와 손실률 출력
    print('Epoch [{}/{}], Train Loss: {:.4f}, Train Acc: {:.2f}%, Test Loss: {:.4f}, Test Acc: {:.2f}%, Time: {:.4f}s'
          .format(epoch + 1, num_epochs, train_loss, train_acc, test_loss, test_acc, time.time() - start_time))
    if test_loss < best_loss:
        best_loss = test_loss
        best_loss_epoch = epoch
        torch.save(model, 'model_bestLoss.pth')
    if test_acc > best_acc:
        best_acc = test_acc
        best_acc_epoch = epoch
        torch.save(model, 'model_bestAcc.pth')
    
    print('best_loss_epoch : {} , best loss: {}'.format(best_loss_epoch, best_loss))
    print('best_acc_epoch : {} , best acc: {}'.format(best_loss_epoch, best_loss))s

#### 학습 결과 시각화(그래프 / 혼동함수)

In [ ]:
# graph

# 정확도 그래프 그리기
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.plot(train_acc_list, label='Train Acc')
plt.plot(test_acc_list, label='Test Acc')
plt.legend() # 범례 추가
plt.show()

# 손실률 그래프 그리기
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(train_loss_list, label='Train Loss')
plt.plot(test_loss_list, label='Test Loss')
plt.legend() # 범례 추가
plt.show()

In [ ]:
# confusion_matrix

# 각 항목별 정확도
# 클래스의 수
num_class = 4
# confusion_matrix를 행/열의 길이를 num_class로 모든 원소가 0인 tensor를 생성 - 예측결과와 실제결과 간의 차이를 표시하기 위한 행렬
confusion_matrix = torch.zeros(num_class, num_class)
# 파라미터 업데이트 없이 forward propagation을 수행
with torch.no_grad():
    # test_loader의 데이터 셋을 반복해서
    for i, (inputs, labels) in enumerate(test_loader):
        # 입력 이미지, 라벨 정보를 GPU를 사용하기 위해 to.device()사용
        inputs, labels = inputs.to(device), labels.to(device)
        # 모델에 이미지 학습 (forward propagation이 이루어지며, 모델은 입력을 받아 출력값을 계산)
        outputs = model(inputs)
        # 출력값(outputs) 중에서 가장 큰 값(_)과 그 값이 존재하는 인덱스(predicted)를 반환
        _, preds = torch.max(outputs, 1)
        # label(정답)과 preds(예측값)를 zip으로 하나로 묶어서 반복한 후
        for t, p in zip(labels.view(-1), preds.view(-1)):
                # confusion_matrix의 t 레이블의 p 인덱스에 1을 추가
                confusion_matrix[t.long(), p.long()] += 1

# class 이름 출력
print('break', 'disappearance', 'good')
# 대각선(TP와 TN)에 해당하는 값을 가져오는 코드 / 각 행의 합을 계산하는 코드(전체 샘플의 수)
print(confusion_matrix.diag()/confusion_matrix.sum(1))

# 컨퓨전 매트릭스 heatmap 그리기 ---------------------------------------------
# 현재 Figure 객체에 서브플롯(subplot)을 추가하고, 그 서브플롯의 Axis 객체를 변수 ax에 할당
ax= plt.subplot()
# confusion_matrix를 가지고 히트맵을 그림(annot=True(히트맵에 각 셀에 값을 표기), fmt='g'(각 셀의 숫자로 표기), ax=ax(그래프로 그릴 서브플롯을 지정))
sns.heatmap(confusion_matrix, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation

# x축과 y축의 라벨 표시
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
# 히트맵의 제목 표시
ax.set_title('Confusion Matrix'); 
# x축과 y축의 눈금 설정
ax.xaxis.set_ticklabels(['break', 'disappearance', 'good', 'loose']); ax.yaxis.set_ticklabels(['break', 'disappearance', 'good', 'loose']);

#### 분류 모델을 평가 - F1 score(모델을 평가하는 지표)

In [ ]:
# F1 score

# 모든 데이터의 예측 결과와 정답을 저장할 리스트
all_preds = []
all_labels = []

# 파라미터 업데이트 없이 forward propagation을 수행
with torch.no_grad():
    # test_loader의 데이터 셋을 반복해서
    for inputs, labels in test_loader:
        # 입력 이미지, 라벨 정보를 GPU를 사용하기 위해 to.device()사용
        inputs, labels = inputs.to(device), labels.to(device)

        # 모델에 이미지 학습 (forward propagation이 이루어지며, 모델은 입력을 받아 출력값을 계산)
        outputs = model(inputs)
        # 모델이 예측한 답을 numpy로 변환 all_preds에 append
        all_preds.append(outputs.argmax(dim=1).cpu().numpy())
        # 실제 정답을 numpy로 변환해서 all_lables에 append
        all_labels.append(labels.cpu().numpy())

# all_preds와 all_labels를 하나의 넘파이 배열 합쳐서 변환
all_preds = np.concatenate(all_preds, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

# 정답과 예측결과로 f1 score를 구함
f1_micro = f1_score(all_labels, all_preds, average='micro')         # 각 클래스의 샘플을 모두 더해 계산
f1_weighted = f1_score(all_labels, all_preds, average='weighted')   # 각 클래스의 샘플 수에 따라 가중치 적용 후 계산
f1_macro = f1_score(all_labels, all_preds, average='macro')         # 각 클래스별로 계산 후 평균 냄

print(f1_micro, f1_weighted , f1_macro)

#### 평가 코드

In [ ]:
# 모델을 평가 모드로 설정
model.eval()

# PyTorch의 연산 그래프에서 이전 계산에서 계산한 gradient를 기록하지 않고 연산을 수행하는 context 
# 테스트 단계에서 모델의 평가를 수행할 때 사용
with torch.no_grad():
    # 각각 전체 loss와 정확도를 누적하는 변수
    # 테스트 데이터셋의 모든 배치를 순회하면서 모델의 예측과 정답을 비교하여 loss와 정확도를 누적
    running_loss = 0
    running_corrects = 0
    running_top3_corrects = 0    
    
    # 배치 단위로 나눈 테스트 데이터 순회하며 불러와서
    for inputs, labels in test_loader:
        # 입력 이미지 데이터, 정답 라벨 / GPU를 사용하기 위해 .to(device) 메소드를 사용
        inputs, labels = inputs.to(device), labels.to(device)

        # 모델에 학습 데이터를 입력 (forward propagation이 이루어지며, 모델은 입력을 받아 출력값을 계산)
        outputs = model(inputs)
        # 출력값(outputs) 중에서 가장 큰 값(_)과 그 값이 존재하는 인덱스(predicted)를 반환
        _, preds = torch.max(outputs, 1)
        
        # 모델이 예측한 출력값 outputs와 실제 정답인 라벨 labels를 비교하여 손실 값을 계산
        loss = criterion(outputs, labels)

        # 이번 배치에서 계산된 손실 값과 정확도를 running_loss와 running_corrects 변수에 더함
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        
        # top3 정확도 계산
        _, top3_preds = torch.topk(outputs, k=3)
        top3_corrects = torch.sum(top3_preds == labels.unsqueeze(1).expand_as(top3_preds))
        running_top3_corrects += top3_corrects
      

        # 해당 이미지를 출력하고 예측결과를 같이 보여줌
        imshow(inputs.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])
        # 이미지의 예측결과와 실제 정답을 출력
        print(f'[예측 결과: {class_names[preds[0]]}, {class_names[preds[1]]}] (실제 정답: {class_names[labels.data[0]]})')
        
        #------클래스별 확률 분포 확인하기-------------
        # softmax 함수를 이용하여 클래스별 확률 분포로 변환
        probs = F.softmax(outputs, dim=1)

        # 출력값 중 가장 높은 값을 가지는 인덱스를 이용하여 예측 결과 추출
        _, preds = torch.max(probs, 1)
 
        # 클래스별 확률 분포 출력
        for i in range(len(class_names)):
            print('{}: {:.2f}%'.format(class_names[i], probs[0, i]*100))
        #-----------------------------------------------
        
    # 모든 배치에서 계산된 손실 값과 정확도를 이용하여 epoch 단위로 평균 손실 값과 정확도를 계산
    epoch_loss = running_loss / len(test_datasets)
    epoch_acc = running_corrects / len(test_datasets) * 100.
    epoch_top3_acc = running_top3_corrects / len(test_datasets) * 100.
    

    # 테스트 과정 중에 결과 출력
    print('[Test Phase] Loss: {:.4f} Acc: {:.4f}% Top3 Acc: {:.4f}% Time: {:.4f}s'.format(epoch_loss, epoch_acc, epoch_top3_acc, time.time() - start_time))
    